In [3]:
!pip install torch==2.2.1+cpu -f https://download.pytorch.org/whl/torch_stable.html
!pip install torch-geometric -f https://data.pyg.org/whl/torch-2.2.1+cpu.html
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.2.1+cpu.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.2.1+cpu.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-2.2.1+cpu.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-2.2.1+cpu.html


Looking in links: https://download.pytorch.org/whl/torch_stable.html
Looking in links: https://data.pyg.org/whl/torch-2.2.1+cpu.html
Looking in links: https://data.pyg.org/whl/torch-2.2.1+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.8/511.8 kB 2.7 MB/s eta 0:00:002.2 MB/s eta 0:00:01
Looking in links: https://data.pyg.org/whl/torch-2.2.1+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 1.3 MB/s eta 0:00:00 MB/s eta 0:00:010m:02
Looking in links: https://data.pyg.org/whl/torch-2.2.1+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.6/776.6 kB 982.1 kB/s eta 0:00:001m954.7 kB/s eta 0:00:01
Looking in links: https://data.pyg.org/whl/torch-2.2.1+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.8/215.8 kB 427.9 kB/s eta 0:00:00 kB/s eta 0:00:01:01


In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from transformers import BertTokenizer, BertModel
import networkx as nx
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.utils import from_networkx

/home/marius/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [8]:
import pandas as pd
import torch
import torch.nn.functional as F
import networkx as nx
from transformers import BertTokenizer, BertModel
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.utils import from_networkx

# Sample CSV data
data = {
    "Timestamp": ["2024-05-15T16:00:15.887Z"],
    "OS": ["win10"],
    "IP": ["fe80::24b4:3691:44a6:38a1"],
    "LogLevel": ["information"],
    "EventID": [5379],
    "Category": ["User Account Management"],
    "Description": [
        """Credential Manager credentials were read.

        Subject:
                Security ID:            S-1-5-18
                Account Name:           WIN10$
                Account Domain:         sec699-20
                Logon ID:               0x3E7
                Read Operation:         Enumerate Credentials

        This event occurs when a user performs a read operation on stored credentials in Credential Manager."""
    ]
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Function to parse hierarchical data from description
def parse_description(description):
    hierarchy = {}
    lines = description.split('\n')
    key = None
    for line in lines:
        line = line.strip()
        if not line:
            continue
        if ':' in line:
            parts = line.split(':', 1)
            key = parts[0].strip()
            value = parts[1].strip()
            hierarchy[key] = value
        elif key:
            hierarchy[key] += ' ' + line.strip()
    return hierarchy

# Apply parsing to the Description column
df['ParsedDescription'] = df['Description'].apply(parse_description)

# Initialize graph
G = nx.Graph()

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Add nodes and edges to the graph
for index, row in df.iterrows():
    parsed = row['ParsedDescription']
    description = row['Description']
    
    # Tokenize and encode the description
    inputs = tokenizer(description, return_tensors='pt', max_length=512, truncation=True, padding='max_length')
    outputs = model(**inputs)
    description_embedding = outputs.last_hidden_state[:, 0, :].detach().numpy().flatten()
    
    # Add event node with BERT embedding
    event_node = 'Event_{}'.format(index)
    G.add_node(event_node, type='event', embedding=description_embedding)
    
    # Add other nodes and connect to event node
    for key, value in parsed.items():
        if not G.has_node(value):
            G.add_node(value, type=key, embedding=torch.zeros(768).numpy())  # Add default embedding for non-event nodes
        G.add_edge(event_node, value)

# Convert NetworkX graph to PyTorch Geometric data
data = from_networkx(G)

# Add node features to the data object
node_embeddings = [G.nodes[node]['embedding'] for node in G.nodes]
data.x = torch.tensor(node_embeddings, dtype=torch.float)

# Define a simple GCN model
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(data.num_node_features, 256)
        self.conv2 = GCNConv(256, 128)
        self.conv3 = GCNConv(128, 768)  # Output size matches the input embedding size

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        return x

# Create and train the model
model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.mse_loss(out, data.x)  # Dummy loss for example purposes
    loss.backward()
    optimizer.step()

# Extract the embeddings from the trained model
model.eval()
with torch.no_grad():
    embeddings = model(data)

# Print the embeddings
print("GCN embeddings shape:", embeddings.shape)
print(embeddings)


GCN embeddings shape: torch.Size([7, 768])
tensor([[-0.2238, -0.1572,  0.0377,  ..., -0.1945, -0.2714,  0.7107],
        [-0.0212, -0.0117,  0.0010,  ..., -0.0172, -0.0238,  0.0644],
        [-0.0212, -0.0117,  0.0010,  ..., -0.0172, -0.0238,  0.0644],
        ...,
        [-0.0212, -0.0117,  0.0010,  ..., -0.0172, -0.0238,  0.0644],
        [-0.0212, -0.0117,  0.0010,  ..., -0.0172, -0.0238,  0.0644],
        [-0.0212, -0.0117,  0.0010,  ..., -0.0172, -0.0238,  0.0644]])
